In [1]:
from projection.laserscan import LaserScan, SemLaserScan
import yaml
import argparse
from torchvision.transforms import transforms
import numpy as np
import matplotlib
from matplotlib import cm
args = argparse.ArgumentParser("test")
args.add_argument("--config", type=str, default= "./config/semantic-kitti.yaml")
args = args.parse_args(args=[])

with open(args.config) as config:
    CFG = yaml.safe_load(config)
color_dict = CFG['color_map']
topil = transforms.ToPILImage()

laserscan = SemLaserScan(color_dict, project=True)
scan_path = "./bin/denserpc.bin"
img_path = scan_path.replace('velodyne', 'image_2')[:-3] + 'png'
label_path = scan_path.replace('velodyne', 'labels')[:-3] + 'label'
laserscan.open_scan(scan_path, scan_unfold=False)
laserscan.open_label(label_path)

# cmap = matplotlib.colormaps.get_cmap('viridis')
cmap = cm.get_cmap('viridis')
sm = cm.ScalarMappable(cmap=cmap)
color_range = sm.to_rgba(np.linspace(0,1,256), bytes=True)[:, 0:3]

range_img = laserscan.proj_range
range_img = ((range_img-range_img.min())/(range_img.max()-range_img.min())*255).astype(np.uint8)
range_img_color = color_range[range_img].astype(np.uint8)

semantic_img = topil((laserscan.proj_sem_color * 255).astype(np.uint8))
range_img = topil(range_img_color)
semantic_img.show()
range_img.show()


FileNotFoundError: [Errno 2] No such file or directory: './denserpc.label'

In [ ]:
laserscan.open_scan(scan_path)
laserscan.open_label(label_path)
range_img = laserscan.proj_range
range_img = ((range_img-range_img.min())/(range_img.max()-range_img.min())*255).astype(np.uint8)
range_img_color = color_range[range_img].astype(np.uint8)

semantic_img = topil((laserscan.proj_sem_color * 255).astype(np.uint8))
range_img = topil(range_img_color)
semantic_img.show()
range_img.show('scan unfold')

In [ ]:
from pcm.proximity_convolution import proximity_conv
from torch.nn import Conv2d
from torch.nn.parameter import Parameter
import torch
totensor = transforms.ToTensor()
p_conv = proximity_conv(1, 1, 3, stride=2)
n_conv = Conv2d(1, 1, 3, stride=2)
n_conv.weight = Parameter(torch.ones(1,1,3,3)/9.)
p_conv_input = totensor(laserscan.proj_range).unsqueeze(0)
n_conv_input = totensor(laserscan.proj_range).unsqueeze(0)
# for i in range(4):
#     p_conv_input = p_conv(p_conv_input)
#     n_conv_input = n_conv(n_conv_input)
p_conv_result = p_conv(p_conv_input)[0].permute(1,2,0).detach().numpy()[:,:,0]
n_conv_result = n_conv(n_conv_input)[0].permute(1,2,0).detach().numpy()[:,:,0]
p_conv_img = ((p_conv_result - p_conv_result.min())/(p_conv_result.max()-p_conv_result.min())*255).astype(np.uint8)
n_conv_img = ((n_conv_result - n_conv_result.min())/(n_conv_result.max()-n_conv_result.min())*255).astype(np.uint8)
p_conv_color = color_range[p_conv_img]
n_conv_color = color_range[n_conv_img]

p_conv_color = topil(p_conv_color)
n_conv_color = topil(n_conv_color)
p_conv_color.show()
n_conv_color.show()


/home/shitong/anaconda3/envs/penet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'transforms' is not defined

In [ ]:
from pcm.proximity_convolution import proximity_conv
from torch.nn import Conv2d
from torch.nn.parameter import Parameter
import torch
totensor = transforms.ToTensor()
p_conv = proximity_conv(3, 3, 3, stride=2)
n_conv = Conv2d(3, 3, 3, stride=2)
n_conv.weight = Parameter(torch.ones(3,3,3,3)/9.)
p_conv_input = totensor(laserscan.proj_sem_color).unsqueeze(0)
n_conv_input = totensor(laserscan.proj_sem_color).unsqueeze(0)
# for i in range(4):
#     p_conv_input = p_conv(p_conv_input)
#     n_conv_input = n_conv(n_conv_input)
p_conv_result = p_conv(p_conv_input)[0]
n_conv_result = n_conv(n_conv_input)[0]

p_conv_color = topil(p_conv_result)
n_conv_color = topil(n_conv_result)
p_conv_color.show()
n_conv_color.show()
